In [1]:
from typing import cast

import numpy as np
import torch
from agent.graph_extractor import GraphExtractor
from environment.environment import Environment
from environment.grid import Grid
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from torch_geometric.nn import summary

In [ ]:
device = torch.device("cuda:3")
rng = np.random.default_rng(0)

grid = Grid(rng=rng)
env = Environment(grid, rng=rng)
# check_env(env)

policy_kwargs = {"features_extractor_class": GraphExtractor}
model = PPO(
    "MultiInputPolicy",
    env,
    policy_kwargs=policy_kwargs,
    verbose=2,
    device=device,
    n_steps=1280,
)


Using cuda:3 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
feature_extractor = cast(GraphExtractor, model.policy.features_extractor)
obs = {
    key: torch.tensor(value, dtype=torch.float32, device=device).unsqueeze(0)
    for key, value in env.observe().items()
}
print(summary(feature_extractor, obs))


+----------------------------+-------------------+----------------+----------+
| Layer                      | Input Shape       | Output Shape   | #Param   |
|----------------------------+-------------------+----------------+----------|
| GraphExtractor             |                   | [1, 640]       | 9,001    |
| ├─(node_type)Embedding     | [1, 10]           | [1, 10, 8]     | 32       |
| ├─(phase)MLP               | [1, 10, 2]        | [1, 10, 8]     | 96       |
| │    └─(mlp)Sequential     | [1, 10, 2]        | [1, 10, 8]     | 96       |
| │    │    └─(0)Linear      | [1, 10, 2]        | [1, 10, 8]     | 24       |
| │    │    └─(1)Identity    | [1, 10, 8]        | [1, 10, 8]     | --       |
| │    │    └─(2)ELU         | [1, 10, 8]        | [1, 10, 8]     | --       |
| │    │    └─(3)Identity    | [1, 10, 8]        | [1, 10, 8]     | --       |
| │    │    └─(4)Linear      | [1, 10, 8]        | [1, 10, 8]     | 72       |
| │    │    └─(5)Identity    | [1, 10, 8]        | [

In [ ]:
model.learn(total_timesteps=1280 * 10)


/pds/pds11/hoyun/swing_rl/SMU_v1/environment/grid.py:472: RuntimeWarning: invalid value encountered in divide
  weights /= np.sum(np.abs(weights))


ValueError: probabilities contain NaN